If needed, install graphviz above

In [1]:
import sys
# gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
# sys.path.append(gems_tco_path)

import GEMS_TCO
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data_local_computer


from collections import defaultdict

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import copy                    # clone tensor

In [2]:
lat_lon_resolution = [10,10]
mm_cond_number = 20
years = ['2024']
month_range =[7,8]
idx_for_datamap= [0,8]

params= [20, 8.25, 5.25, 0.2, 0.5, 5]

instance = load_data_local_computer()
map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap=[0,8])


load data

# Likelihood check

In [4]:
# Define your initial parameters
params = [21.8, 1.09, 1.17, 0.2, .2, 0.5, 1]
params = torch.tensor(params, requires_grad=True)

torch_smooth = torch.tensor(0.5, dtype=torch.float32)

instance = kernels.likelihood_function(smooth=torch_smooth , input_map=analysis_data_map,aggregated_data=aggregated_data, nns_map=nns_map, mm_cond_number=mm_cond_number)

out0 = instance.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_anisotropy_v05)
print(out0)

out0 = instance.vecchia_extrapolate(params, instance.matern_cov_anisotropy_v05)
print(out0)


tensor(2675.2982, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(2664.4289, dtype=torch.float64, grad_fn=<AddBackward0>)


In [5]:
def matern_cov_yx( params: torch.Tensor, y: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    sigmasq, range_, range_lon, advec, beta, nugget = params
    if y is None or x is None:
        raise ValueError("Both y and x_df must be provided.")

    x1 = x[:, 0]
    y1 = x[:, 1]
    t1 = x[:, 3]

    x2 = y[:, 0]
    y2 = y[:, 1]
    t2 = y[:, 3]

    spat_coord1 = torch.stack((x1 - advec * t1, y1 - advec * t1), dim=-1)
    spat_coord2 = torch.stack((x2 - advec * t2, y2 - advec * t2), dim=-1)

    coords1 = torch.cat((spat_coord1, (beta * t1).reshape(-1, 1)), dim=1)
    coords2 = torch.cat((spat_coord2, (beta * t2).reshape(-1, 1)), dim=1)

  
    def custom_distance_matrix(U, V):
        diff = U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0)
        spatial_diff = torch.norm(diff, dim=2)
        temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
        return torch.sqrt(spatial_diff**2 + temporal_diff**2)

    distance = custom_distance_matrix(coords1, coords2)
    out = torch.zeros_like(distance)

    non_zero_indices = distance != 0
    if torch.any(non_zero_indices):
        out[non_zero_indices] = sigmasq * torch.exp(-distance[non_zero_indices]/range_)
    out[~non_zero_indices] = sigmasq

    # Add a small jitter term to the diagonal for numerical stability
    out += torch.eye(out.shape[0]) * nugget
    return out

# to track gradient, torch sqrt in distance is moved 
def matern_cov_yx2( params: torch.Tensor, y: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    sigmasq, range_, range_lon, advec, beta, nugget = params
    if y is None or x is None:
        raise ValueError("Both y and x_df must be provided.")

    x1 = x[:, 0]
    y1 = x[:, 1]
    t1 = x[:, 3]

    x2 = y[:, 0]
    y2 = y[:, 1]
    t2 = y[:, 3]

    spat_coord1 = torch.stack((x1 - advec * t1, y1 - advec * t1), dim=-1)
    spat_coord2 = torch.stack((x2 - advec * t2, y2 - advec * t2), dim=-1)

    coords1 = torch.cat((spat_coord1, (beta * t1).reshape(-1, 1)), dim=1)
    coords2 = torch.cat((spat_coord2, (beta * t2).reshape(-1, 1)), dim=1)

    U= coords1
    V = coords2 
 
    diff = U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0)
    spatial_diff = torch.norm(diff, dim=2)
    temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
    distance = (spatial_diff**2 + temporal_diff**2)

    out = torch.zeros_like(distance)

    non_zero_indices = distance != 0
    if torch.any(non_zero_indices):
        out[non_zero_indices] = sigmasq * torch.exp(- torch.sqrt(distance[non_zero_indices])/range_)
    out[~non_zero_indices] = sigmasq

    # Add a small jitter term to the diagonal for numerical stability
    out += torch.eye(out.shape[0]) * nugget

    return out


def full_likelihood( params: torch.Tensor, input_np: torch.Tensor, y: torch.Tensor, covariance_function) -> torch.Tensor:
    input_arr = input_np[:, :4]
    y_arr = y

    # Compute the covariance matrix
    cov_matrix = covariance_function(params=params, y=input_arr, x=input_arr)
    
    # Compute the log determinant of the covariance matrix
    sign, log_det = torch.slogdet(cov_matrix)
    # if sign <= 0:
    #     raise ValueError("Covariance matrix is not positive definite")
    
    # Extract locations
    locs = input_arr[:, :2]

    # Compute beta
    tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
    tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
    beta = torch.linalg.solve(tmp1, tmp2)

    # Compute the mean
    mu = torch.matmul(locs, beta)
    y_mu = y_arr - mu

    # Compute the quadratic form
    quad_form = torch.matmul(y_mu, torch.linalg.solve(cov_matrix, y_mu))

    # Compute the negative log likelihood
    neg_log_lik = 0.5 * (log_det + quad_form)

    return log_det, quad_form, neg_log_lik
params = [20, 8.25, 5.25, 0.2, 0.5, 5]
params = torch.tensor(params, requires_grad=True)

def matern_partial_sigma( params: torch.Tensor, y: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    sigmasq, range_, range_lon, advec, beta, nugget = params
    if y is None or x is None:
        raise ValueError("Both y and x_df must be provided.")

    x1 = x[:, 0]
    y1 = x[:, 1]
    t1 = x[:, 3]

    x2 = y[:, 0]
    y2 = y[:, 1]
    t2 = y[:, 3]

    spat_coord1 = torch.stack((x1 - advec * t1, y1 - advec * t1), dim=-1)
    spat_coord2 = torch.stack((x2 - advec * t2, y2 - advec * t2), dim=-1)

    coords1 = torch.cat((spat_coord1, (beta * t1).reshape(-1, 1)), dim=1)
    coords2 = torch.cat((spat_coord2, (beta * t2).reshape(-1, 1)), dim=1)

    def custom_distance_matrix(U, V):
        diff = U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0)
        spatial_diff = torch.norm(diff, dim=2)
        temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
        return torch.sqrt(spatial_diff**2 + temporal_diff**2)

    distance = custom_distance_matrix(coords1, coords2)
    out = torch.zeros_like(distance)

    non_zero_indices = distance != 0
    if torch.any(non_zero_indices):
        out[non_zero_indices] = torch.exp(-distance[non_zero_indices]/range_)
    out[~non_zero_indices] = 1

    return out

def matern_partial_range( params: torch.Tensor, y: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    sigmasq, range_, range_lon, advec, beta, nugget = params
    if y is None or x is None:
        raise ValueError("Both y and x_df must be provided.")

    x1 = x[:, 0]
    y1 = x[:, 1]
    t1 = x[:, 3]

    x2 = y[:, 0]
    y2 = y[:, 1]
    t2 = y[:, 3]
 
    spat_coord1 = torch.stack((x1 - advec * t1, y1 - advec * t1), dim=-1)
    spat_coord2 = torch.stack((x2 - advec * t2, y2 - advec * t2), dim=-1)

    coords1 = torch.cat((spat_coord1, (beta * t1).reshape(-1, 1)), dim=1)
    coords2 = torch.cat((spat_coord2, (beta * t2).reshape(-1, 1)), dim=1)

    def custom_distance_matrix(U, V):
        diff = U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0)
        spatial_diff = torch.norm(diff, dim=2)
        temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
        return torch.sqrt(spatial_diff**2 + temporal_diff**2)

    distance = custom_distance_matrix(coords1, coords2)
    out = torch.zeros_like(distance)

    non_zero_indices = distance != 0
    if torch.any(non_zero_indices):
        out[non_zero_indices] = sigmasq * torch.exp(-distance[non_zero_indices] / range_)
    out[~non_zero_indices] = sigmasq

    # Compute the derivative with respect to the range parameter
    derivative = torch.zeros_like(distance)
    if torch.any(non_zero_indices):
        derivative[non_zero_indices] = out[non_zero_indices] * distance[non_zero_indices] / range_**2

    # Ensure the derivative on the diagonal is zero
    derivative.fill_diagonal_(0)
    
    return derivative

def matern_partial_nugget( params: torch.Tensor, y: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    sigmasq, range_, range_lon, advec, beta, nugget = params
    if y is None or x is None:
        raise ValueError("Both y and x_df must be provided.")

    x1 = x[:, 0]
    y1 = x[:, 1]
    t1 = x[:, 3]

    x2 = y[:, 0]
    y2 = y[:, 1]
    t2 = y[:, 3]

    spat_coord1 = torch.stack((x1 - advec * t1, y1 - advec * t1), dim=-1)
    spat_coord2 = torch.stack((x2 - advec * t2, y2 - advec * t2), dim=-1)

    coords1 = torch.cat((spat_coord1, (beta * t1).reshape(-1, 1)), dim=1)
    coords2 = torch.cat((spat_coord2, (beta * t2).reshape(-1, 1)), dim=1)

    def custom_distance_matrix(U, V):
        diff = U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0)
        spatial_diff = torch.norm(diff, dim=2)
        temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
        return torch.sqrt(spatial_diff**2 + temporal_diff**2)

    distance = custom_distance_matrix(coords1, coords2)
    out = torch.zeros_like(distance)

    non_zero_indices = distance != 0
    if torch.any(non_zero_indices):
        out[non_zero_indices] = 0
    out[~non_zero_indices] = 1

    return out

def matern_partial_beta( params: torch.Tensor, y: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    sigmasq, range_, range_lon, advec, beta, nugget = params
    if y is None or x is None:
        raise ValueError("Both y and x_df must be provided.")

    x1 = x[:, 0]
    y1 = x[:, 1]
    t1 = x[:, 3]

    x2 = y[:, 0]
    y2 = y[:, 1]
    t2 = y[:, 3]

    spat_coord1 = torch.stack((x1 - advec * t1, y1 - advec * t1), dim=-1)
    spat_coord2 = torch.stack((x2 - advec * t2, y2 - advec * t2), dim=-1)

    coords1 = torch.cat((spat_coord1, (beta * t1).reshape(-1, 1)), dim=1)
    coords2 = torch.cat((spat_coord2, (beta * t2).reshape(-1, 1)), dim=1)

    def custom_distance_matrix(U, V):
        diff = U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0)
        spatial_diff = torch.norm(diff, dim=2)
        temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
        return torch.sqrt(spatial_diff**2 + temporal_diff**2)


    distance = custom_distance_matrix(coords1, coords2)
    out = torch.zeros_like(distance)

    non_zero_indices = distance != 0
    if torch.any(non_zero_indices):
        tmp1 = sigmasq * torch.exp(-distance[non_zero_indices]/range_)*(-1/range_)
        
        tmp2 = (beta/distance[non_zero_indices])

                # Adjust t2 - t1 for non-zero indices
        t1_non_zero = t1.unsqueeze(1).expand_as(distance)[non_zero_indices]
        t2_non_zero = t2.unsqueeze(0).expand_as(distance)[non_zero_indices]

        out[non_zero_indices] = tmp1 * tmp2 *(t2_non_zero - t1_non_zero)**2

    out[~non_zero_indices] = 0

    return out

def matern_partial_advec( params: torch.Tensor, y: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    sigmasq, range_, range_lon, advec, beta, nugget = params
    if y is None or x is None:
        raise ValueError("Both y and x_df must be provided.")

    x1 = x[:, 0]
    y1 = x[:, 1]
    t1 = x[:, 3]

    x2 = y[:, 0]
    y2 = y[:, 1]
    t2 = y[:, 3]

    spat_coord1 = torch.stack((x1 - advec * t1, y1 - advec * t1), dim=-1)
    spat_coord2 = torch.stack((x2 - advec * t2, y2 - advec * t2), dim=-1)

    coords1 = torch.cat((spat_coord1, (beta * t1).reshape(-1, 1)), dim=1)
    coords2 = torch.cat((spat_coord2, (beta * t2).reshape(-1, 1)), dim=1)

    def custom_distance_matrix(U, V):
        diff = U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0)
        spatial_diff = torch.norm(diff, dim=2)
        temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
        return torch.sqrt(spatial_diff**2 + temporal_diff**2)


    distance = custom_distance_matrix(coords1, coords2)
    out = torch.zeros_like(distance)

    non_zero_indices = distance != 0
    if torch.any(non_zero_indices):
        tmp1 = sigmasq * torch.exp(-distance[non_zero_indices]/range_)*(-1/range_)

        # Adjust t2 - t1 for non-zero indices
        t1_non_zero = t1.unsqueeze(1).expand_as(distance)[non_zero_indices]
        t2_non_zero = t2.unsqueeze(0).expand_as(distance)[non_zero_indices]
        x1_non_zero = x1.unsqueeze(1).expand_as(distance)[non_zero_indices]
        x2_non_zero = x2.unsqueeze(0).expand_as(distance)[non_zero_indices]
        y1_non_zero = y1.unsqueeze(1).expand_as(distance)[non_zero_indices]
        y2_non_zero = y2.unsqueeze(0).expand_as(distance)[non_zero_indices]

        tmp2 = (t2_non_zero - t1_non_zero )
        tmp3 = tmp2* ( (x1_non_zero-x2_non_zero) - advec*(t1_non_zero - t2_non_zero) +  (y1_non_zero-y2_non_zero) - advec*(t1_non_zero - t2_non_zero) )

        out[non_zero_indices] = tmp1* (1/ distance[non_zero_indices]) * tmp3

    out[~non_zero_indices] = 0
    return out

def full_partial( params: torch.Tensor, input_np: torch.Tensor, y: torch.Tensor, covariance_function, partial_fun) -> torch.Tensor:
    input_arr = input_np[:, :4]
    y_arr = y

    # Compute the covariance matrix
    cov_matrix = covariance_function(params=params, y=input_arr, x=input_arr)
    partial_der = partial_fun(params=params, y=input_arr, x=input_arr)
    
    # Compute the log determinant of the covariance matrix
    # sign, log_det = torch.slogdet(cov_matrix)
    # if sign <= 0:
    #     raise ValueError("Covariance matrix is not positive definite")
    
    # Extract locations
    locs = input_arr[:, :2]

    # Compute beta
    tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
    tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
    beta = torch.linalg.solve(tmp1, tmp2)

    # Compute the mean
    mu = torch.matmul(locs, beta)
    y_mu = y_arr - mu

    # Compute the quadratic form
    tmp1 = torch.linalg.solve(cov_matrix, partial_der)
    tmp2 = torch.matmul(y_mu, tmp1)
    tmp3 = torch.linalg.solve(cov_matrix, y_mu)
    quad_form =  -torch.matmul(tmp2,tmp3 )
    log_det = torch.trace(tmp1)
    # Compute the negative log likelihood
    neg_log_lik = 0.5 * (log_det + quad_form)
    
    return log_det, quad_form, neg_log_lik

params = [20, 8.25, 5.25, 0.2, 0.5, 0.5]
params = torch.tensor(params, requires_grad=True)



aggregated_np = aggregated_data
a,b,c = full_partial(params, aggregated_np[:,:4], aggregated_np[:,2], matern_cov_yx, matern_partial_sigma)
print(f'sigma log_det:{a}')
print(f'sigma quad_form:{b}')
print(f'sigma quad_form:{c}')

a,b,c = full_partial(params, aggregated_np[:,:4], aggregated_np[:,2], matern_cov_yx, matern_partial_range)
print(f'\nrange log_det:{a}')
print(f'range quad_form:{b}')
print(f'range quad_form:{c}')

a,b,c = full_partial(params, aggregated_np[:,:4], aggregated_np[:,2], matern_cov_yx, matern_partial_nugget)
print(f'\nnugget log_det:{a}')
print(f'nugget quad_form:{b}')
print(f'nugget quad_form:{c}')

matern_partial_sigma(params, aggregated_np[:,:4], aggregated_np[:,:4])
a,b,c = full_partial(params, aggregated_np[:,:4], aggregated_np[:,2], matern_cov_yx, matern_partial_beta)
print(f'\nbeta log_det:{a}')
print(f'beta quad_form:{b}')
print(f'beta quad_form:{c}')

matern_partial_sigma(params, aggregated_np[:,:4], aggregated_np[:,:4])
a,b,c = full_partial(params, aggregated_np[:,:4], aggregated_np[:,2], matern_cov_yx, matern_partial_advec)
print(f'\nadvec log_det:{a}')
print(f'advec quad_form:{b}')
print(f'advec quad_form:{c}')

sigma log_det:52.751060485839844
sigma quad_form:-279.7126770019531
sigma quad_form:-113.48080444335938

range log_det:-126.85018920898438
range quad_form:672.7844848632812
range quad_form:272.9671630859375

nugget log_det:1089.956298828125
nugget quad_form:-6223.43359375
nugget quad_form:-2566.73876953125

beta log_det:599.5950927734375
beta quad_form:-1744.1077880859375
beta quad_form:-572.25634765625

advec log_det:61.72618865966797
advec quad_form:1965.310302734375
advec quad_form:1013.5182495117188


## Great circle distance vs Euclidean distance 
   
In small area, the difference between great circle distance (gcd) and euclidean distance should be small. However, model can try to account for curvature in small area resulting in overfitting. Haversine formula might cause numerical errors because it involves sin and cos for small distances. Also, incompatability between gcd for space and euclidean for time might be an issue.

Based on my experiment, likelihood was way higher when using euclidean distance for space. 


def custom_distance_matrix(self, U, V):
    # Efficient distance computation with broadcasting
    spatial_diff = torch.norm(U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0), dim=2)

    temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
    distance = (spatial_diff**2 + temporal_diff**2)  # move torch.sqrt to covariance function to track gradients of beta and avec
    return distance

#################################################################################
#################################################################################

def great_circle_distance(self, lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(torch.deg2rad, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = torch.sin(dlat / 2)**2 + torch.cos(lat1) * torch.cos(lat2) * torch.sin(dlon / 2)**2
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1 - a))
    
    # Radius of Earth in kilometers (mean radius)
    R = 6371.0
    return R * c
def custom_distance_matrix(self, U, V):
    # Efficient distance computation with broadcasting
    lat1, lon1 = U[:, 0].unsqueeze(1), U[:, 1].unsqueeze(1)
    lat2, lon2 = V[:, 0].unsqueeze(0), V[:, 1].unsqueeze(0)
    
    spatial_diff = self.great_circle_distance(lat1, lon1, lat2, lon2)
    temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
    distance = (spatial_diff**2 + temporal_diff**2)  # move torch.sqrt to covariance function to track gradients of beta and avec
    return distance

## Class using analytical derivatives

In [6]:
class matern_advec_beta:
    def __init__(self, aggregated_np: torch.Tensor, params: torch.Tensor):
        self.aggregated_np = aggregated_np[:,:4]
        self.y = aggregated_np[:,2]
        self.aggregated_locs = aggregated_np[:,:2]
        aggregated_np_clone = copy.deepcopy(aggregated_np)
        self.x1 = aggregated_np[:,0]
  
        self.y1 = aggregated_np[:,1]
        self.t1 = aggregated_np[:,3]

        self.x2 = aggregated_np_clone[:,0]
        self.y2 = aggregated_np_clone[:,1]
        self.t2 = aggregated_np_clone[:,3] 
        self.params = params 

        self.distance, self.non_zero_indices = self.precompute_coords(params)
                # Adjust t2 - t1 for non-zero indices
        self.t1_non_zero = self.t1.unsqueeze(1).expand_as(self.distance)[self.non_zero_indices]
        self.t2_non_zero = self.t2.unsqueeze(0).expand_as(self.distance)[self.non_zero_indices]
        self.x1_non_zero = self.x1.unsqueeze(1).expand_as(self.distance)[self.non_zero_indices]
        self.x2_non_zero = self.x2.unsqueeze(0).expand_as(self.distance)[self.non_zero_indices]
        self.y1_non_zero = self.y1.unsqueeze(1).expand_as(self.distance)[self.non_zero_indices]
        self.y2_non_zero = self.y2.unsqueeze(0).expand_as(self.distance)[self.non_zero_indices]
    
    def custom_distance_matrix(self, U, V):
        # Efficient distance computation with broadcasting
        spatial_diff = torch.norm(U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0), dim=2)
        temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
        distance = torch.sqrt(spatial_diff**2 + temporal_diff**2)  # Keeping torch.sqrt for proper gradients
        return distance

    def precompute_coords(self, params)-> torch.Tensor:
        igmasq, range_,  advec, beta, nugget = params
        # spat_coord1 = torch.stack((self.x1 , self.y1 - advec * self.t1), dim=-1)
        spat_coord1 = torch.stack((self.x1 - advec * self.t1, self.y1 ), dim=-1)
        spat_coord2 = torch.stack((self.x2 - advec * self.t2, self.y2 ), dim=-1)

        U = torch.cat((spat_coord1, (beta * self.t1).reshape(-1, 1)), dim=1)
        V = torch.cat((spat_coord2, (beta * self.t2).reshape(-1, 1)), dim=1)

        distance = self.custom_distance_matrix(U,V)
        non_zero_indices = distance != 0
        return distance, non_zero_indices

    def matern_advec_beta_cov(self,params) -> torch.Tensor:
        sigmasq, range_,  advec, beta, nugget = params
        
        out = torch.zeros_like(self.distance)

        out[self.non_zero_indices] = sigmasq * torch.exp(- (self.distance[self.non_zero_indices])/range_)
        out[~self.non_zero_indices] = sigmasq

        # Add a small jitter term to the diagonal for numerical stability
        out += torch.eye(out.shape[0]) * nugget

        return out

    def matern_partial_sigma(self, params) -> torch.Tensor:
        sigmasq, range_, advec, beta, nugget = params
        out = torch.zeros_like(self.distance)

        # Cache the non-zero indices distance
        non_zero_distance = self.distance[self.non_zero_indices]

        # Compute the exponential term
        exp_term = torch.exp_(-non_zero_distance / range_)

        # Assign the computed values to the output tensor
        out[self.non_zero_indices] = exp_term
        out[~self.non_zero_indices] = 1

        return out

    def matern_partial_range(self, params: torch.Tensor ) -> torch.Tensor:
        sigmasq, range_,  advec, beta, nugget = params

        out = torch.zeros_like(self.distance)
    
        tmp = sigmasq * torch.exp(-(self.distance[self.non_zero_indices]) / range_) 
        out[self.non_zero_indices] = tmp * (self.distance[self.non_zero_indices])  / range_**2

        return out
    def matern_partial_nugget(self, params: torch.Tensor) -> torch.Tensor:
        sigmasq, range_,  advec, beta, nugget = params

        out = torch.zeros_like(self.distance)

   
        out[self.non_zero_indices] = 0
        out[~self.non_zero_indices] = 1

        return out

    def matern_partial_advec(self, params: torch.Tensor) -> torch.Tensor:
        sigmasq, range_,  advec, beta, nugget = params


        out = torch.zeros_like(self.distance)

        tmp1 = sigmasq * torch.exp(-(self.distance[self.non_zero_indices]) /range_)*(-1/range_)


        tmp2 = (self.t2_non_zero - self.t1_non_zero )
        tmp3 = tmp2* ( (self.x1_non_zero-self.x2_non_zero) - advec*(self.t1_non_zero - self.t2_non_zero) +  (self.y1_non_zero-self.y2_non_zero) - advec*(self.t1_non_zero - self.t2_non_zero) )

        out[self.non_zero_indices] = tmp1* (1/ (self.distance[self.non_zero_indices]) ) * tmp3

        out[~self.non_zero_indices] = 0
        return out
    
    def matern_partial_beta(self, params: torch.Tensor) -> torch.Tensor:
        sigmasq, range_,  advec, beta, nugget = params
    

        out = torch.zeros_like(self.distance)

        tmp1 = sigmasq * torch.exp(-(self.distance[self.non_zero_indices])/range_)*(-1/range_)
        
        tmp2 = (beta/(self.distance[self.non_zero_indices]))


        out[self.non_zero_indices] = tmp1 * tmp2 *(self.t2_non_zero - self.t1_non_zero)**2

        out[~self.non_zero_indices] = 0

        return out
    

    def cov_and_inv(self, params):
        cov_matrix = self.matern_advec_beta_cov( params=params)
        cov_inv = torch.linalg.inv(cov_matrix)
                
        # Compute beta
        tmp1 = torch.matmul(self.aggregated_locs.T, torch.linalg.solve(cov_matrix, self.aggregated_locs))
        tmp2 = torch.matmul(self.aggregated_locs.T, torch.linalg.solve(cov_matrix, self.y))
        beta = torch.linalg.solve(tmp1, tmp2)

        # Compute the mean
        mu = torch.matmul(self.aggregated_locs, beta)
        y_mu = self.y - mu
        tmp3 = torch.matmul(cov_inv, y_mu)
        return cov_matrix, cov_inv, mu, tmp3
    
    def full_partial(self, params: torch.Tensor, partial_der_fun, cov_matrix, cov_inv,mu, tmp3) -> torch.Tensor:
        

        partial_der = partial_der_fun(params)
        
        # Compute the log determinant of the covariance matrix
        # sign, log_det = torch.slogdet(cov_matrix)
        # if sign <= 0:
        #     raise ValueError("Covariance matrix is not positive definite")
        
        y_mu = self.y - mu

        # Compute the quadratic form derivative
        # tmp1 = torch.linalg.solve(cov_matrix, partial_der)
        tmp1 = torch.matmul( cov_inv, partial_der)

        tmp2 = torch.matmul(y_mu, tmp1)

        # tmp3 = torch.matmul(cov_inv, y_mu)
        # tmp3 = torch.linalg.solve(cov_matrix, y_mu)

        quad_form_derivative  =  -torch.matmul(tmp2,tmp3 )
        log_det_derivative = torch.trace(tmp1)
        # Compute the negative log likelihood
        neg_log_lik_derivative = 0.5 * (log_det_derivative  + quad_form_derivative )
        
        return log_det_derivative , quad_form_derivative , neg_log_lik_derivative 
    
    def full_likelihood(self, params, covariance_function ,cov_matrix, cov_inv, mu) -> torch.Tensor:
        input_arr = self.aggregated_np
        y_arr = self.y

        # Compute the covariance matrix
        #cov_matrix = covariance_function(params=params)
        
        # Compute the log determinant of the covariance matrix
        sign, log_det = torch.slogdet(cov_matrix)
        # if sign <= 0:
        #     raise ValueError("Covariance matrix is not positive definite")
        
        # Extract locations
        locs = input_arr[:, :2]

        # Compute beta
        # tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
        # tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
        # beta = torch.linalg.solve(tmp1, tmp2)

        # Compute the mean
        # mu = torch.matmul(locs, beta)
        y_mu = y_arr - mu

        # Compute the quadratic form
        # quad_form = torch.matmul(y_mu, torch.linalg.solve(cov_matrix, y_mu))
        tmp = torch.matmul( cov_inv, y_mu)
        quad_form = torch.matmul( y_mu, tmp)

        # Compute the negative log likelihood
        neg_log_lik = 0.5 * (log_det + quad_form)

        return neg_log_lik
    
    def matern_advec_beta_optim(self, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8, epochs=10, tol=1e-3):
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        params = self.params.to(device)  # Assuming params is a tensor and moving it to the device
        
        # Initialize tensors and move to GPU if available
        m = torch.zeros_like(params, device=device)
        v = torch.zeros_like(params, device=device)
        
        b1 = torch.tensor(beta1, device=device)
        b2 = torch.tensor(beta2, device=device)

        prev_neg_log_lik = float('inf')
        
        for epoch in range(epochs):
            self.distance, self.non_zero_indices = self.precompute_coords(params)

            if epoch ==0:
                cov_matrix, cov_inv, mu, tmp3 = self.cov_and_inv(params)
                
            _, _, g_sigma = self.full_partial(params, self.matern_partial_sigma,cov_matrix, cov_inv, mu, tmp3)
            _, _, g_range = self.full_partial(params, self.matern_partial_range,cov_matrix, cov_inv, mu, tmp3)
            _, _, g_advec = self.full_partial(params, self.matern_partial_advec,cov_matrix, cov_inv, mu, tmp3)
            _, _, g_beta = self.full_partial(params, self.matern_partial_beta,cov_matrix, cov_inv, mu, tmp3)
            _, _, g_nugget = self.full_partial(params, self.matern_partial_nugget,cov_matrix, cov_inv, mu, tmp3)

            g = torch.stack([g_sigma, g_range, g_advec, g_beta, g_nugget]).to(device)

            # Update m and v
            m = b1 * m + (1 - b1) * g
            v = b2 * v + (1 - b2) * g**2

            # Bias correction
            m_hat = m / (1 - b1**(epoch + 1))
            v_hat = v / (1 - b2**(epoch + 1))

            # Update params
            params = params - lr * m_hat / ( torch.sqrt(v_hat) + epsilon)

            cov_matrix, cov_inv, mu, tmp3 = self.cov_and_inv(params)
            # Compute the current negative log likelihood
            neg_log_lik = self.full_likelihood(params, self.matern_advec_beta_cov,cov_matrix, cov_inv, mu)
            
            # Print params and likelihood every 10th epoch
            if epoch % 10 == 0:
                print(f"Epoch {epoch}: loss: {neg_log_lik}, \nParams = {params} ")

            # Check for convergence
            if abs(prev_neg_log_lik - neg_log_lik) < tol:
                print(f"Converged at epoch {epoch}")
                break
            
            prev_neg_log_lik = neg_log_lik
        return params


params = [24.56, 2.8, -.0138, .301, 2.12]
params = torch.tensor(params, requires_grad=True)

instance = matern_advec_beta(aggregated_data, params)
epochs=300
instance.matern_advec_beta_optim(lr=0.01,beta1=0.9, beta2= 0.8, epsilon=1e-8, epochs=epochs)



Epoch 0: loss: 2623.149169921875, 
Params = tensor([ 2.4570e+01,  2.7900e+00, -2.3800e-02,  3.1100e-01,  2.1300e+00],
       grad_fn=<SubBackward0>) 
Epoch 10: loss: 2619.02294921875, 
Params = tensor([24.6709,  2.6892, -0.1164,  0.3707,  2.2314], grad_fn=<SubBackward0>) 
Epoch 20: loss: 2604.1767578125, 
Params = tensor([24.7736,  2.5867, -0.0954,  0.3141,  2.3359], grad_fn=<SubBackward0>) 
Epoch 30: loss: 2590.366943359375, 
Params = tensor([24.8776,  2.4836, -0.0818,  0.3159,  2.4410], grad_fn=<SubBackward0>) 
Epoch 40: loss: 2584.289794921875, 
Params = tensor([24.9852,  2.3767, -0.0931,  0.3201,  2.5511], grad_fn=<SubBackward0>) 
Epoch 50: loss: 2573.384521484375, 
Params = tensor([25.0945,  2.2679, -0.0726,  0.2713,  2.6639], grad_fn=<SubBackward0>) 
Epoch 60: loss: 2574.65869140625, 
Params = tensor([25.2047,  2.1579, -0.1163,  0.3090,  2.7767], grad_fn=<SubBackward0>) 
Epoch 70: loss: 2564.40185546875, 
Params = tensor([25.3211,  2.0373, -0.0720,  0.2774,  2.9014], grad_fn=<Sub

: 

# Class for pytorch + vecchia approximation

In [3]:
idx_for_datamap = [0,8]

class matern_advec_beta_torch_vecchia:
    def __init__(self, analaysis_data_map: torch.Tensor, params: torch.Tensor, nns_map=nns_map, mm_cond_number=mm_cond_number):
        self.key_list = sorted(analysis_data_map)
        self.input_map = analysis_data_map

        self.mm_cond_number = mm_cond_number
        self.nns_map = nns_map 
        self.analaysis_data_map = analaysis_data_map
        self.smooth = 0.5  
        sample_df = analaysis_data_map[self.key_list[0]]

        self.size_per_hour = len(sample_df)

    def custom_distance_matrix(self, U, V):
        # Efficient distance computation with broadcasting
        spatial_diff = torch.norm(U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0), dim=2)

        temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
        distance = (spatial_diff**2 + temporal_diff**2)  # move torch.sqrt to covariance function to track gradients of beta and avec
        return distance
    
    def precompute_coords_ani(self, params, y: torch.Tensor, x: torch.Tensor)-> torch.Tensor:
        sigmasq, range_lat, range_lon, advec_lat, advec_lon, beta, nugget = params

        if y is None or x is None:
            raise ValueError("Both y and x_df must be provided.")

        x1 = x[:, 0]
        y1 = x[:, 1]
        t1 = x[:, 3]

        x2 = y[:, 0]
        y2 = y[:, 1]
        t2 = y[:, 3]

        # spat_coord1 = torch.stack((self.x1 , self.y1 - advec * self.t1), dim=-1)
        spat_coord1 = torch.stack(( (x1 - advec_lat * t1)/range_lat, (y1 - advec_lon * t1)/range_lon ), dim=-1)
        spat_coord2 = torch.stack(( (x2 - advec_lat * t2)/range_lat, (y2 - advec_lon * t2)/range_lon ), dim=-1)

        U = torch.cat((spat_coord1, (beta * t1).reshape(-1, 1)), dim=1)
        V = torch.cat((spat_coord2, (beta * t2).reshape(-1, 1)), dim=1)

        distance = self.custom_distance_matrix(U,V)
        non_zero_indices = distance != 0
        return distance, non_zero_indices
    
    # anisotropic in three 
    def matern_cov_ani(self,params: torch.Tensor, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        sigmasq, range_lat, range_lon, advec_lat, advec_lon, beta, nugget = params
        

        distance, non_zero_indices = self.precompute_coords_ani(params, x,y)
        out = torch.zeros_like(distance)

        non_zero_indices = distance != 0
        if torch.any(non_zero_indices):
            out[non_zero_indices] = sigmasq * torch.exp(- torch.sqrt(distance[non_zero_indices]))
        out[~non_zero_indices] = sigmasq

        # Add a small jitter term to the diagonal for numerical stability
        out += torch.eye(out.shape[0]) * nugget

        return out
    
    def full_likelihood(self,params: torch.Tensor, input_np: torch.Tensor, y: torch.Tensor, covariance_function) -> torch.Tensor:
        input_arr = input_np[:, :4]
        y_arr = y

        # Compute the covariance matrix
        cov_matrix = covariance_function(params=params, y=input_arr, x=input_arr)
        
        # Compute the log determinant of the covariance matrix
        sign, log_det = torch.slogdet(cov_matrix)
        # if sign <= 0:
        #     raise ValueError("Covariance matrix is not positive definite")
        
        # Extract locations
        locs = input_arr[:, :2]

        # Compute beta
        tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
        tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
        beta = torch.linalg.solve(tmp1, tmp2)

        # Compute the mean
        mu = torch.matmul(locs, beta)
        y_mu = y_arr - mu

        # Compute the quadratic form
        quad_form = torch.matmul(y_mu, torch.linalg.solve(cov_matrix, y_mu))

        # Compute the negative log likelihood
        neg_log_lik = 0.5 * (log_det + quad_form)
     
        return  neg_log_lik 
    
    def vecchia_like_local_computer(self, params: torch.Tensor, covariance_function) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        
        for time_idx in range(len(self.analaysis_data_map)):
            current_np = self.analaysis_data_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            cur_heads = current_np[:21, :]
            neg_log_lik += self.full_likelihood(params, cur_heads, cur_heads[:, 2], covariance_function)

            for index in range(21, self.size_per_hour):

                
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors)
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 1:
                    cov_matrix = self.cov_map[index]['cov_matrix']
                    tmp_for_beta = self.cov_map[index]['tmp_for_beta']
                    cov_xx_inv = self.cov_map[index]['cov_xx_inv']
                    L_inv = self.cov_map[index]['L_inv']
                    cov_ygivenx = self.cov_map[index]['cov_ygivenx']
                    cond_mean_tmp = self.cov_map[index]['cond_mean_tmp']
                    log_det = self.cov_map[index]['log_det']
                    locs = self.cov_map[index]['locs']
                    
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                    if data_list:
                        conditioning_data = torch.vstack(data_list)
                    else:
                        conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                    np_arr = torch.vstack((current_row, conditioning_data))
                    y_and_neighbors = np_arr[:, 2]

                    cov_yx = cov_matrix[0, 1:]

                    tmp2 = torch.matmul(torch.matmul(L_inv, locs).T, torch.matmul(L_inv, y_and_neighbors))
                    beta = torch.linalg.solve(tmp_for_beta, tmp2)

                    mu = torch.matmul(locs, beta)
                    mu_current = mu[0]
                    mu_neighbors = mu[1:]
                    
                    # Mean and variance of y|x
                    cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                    alpha = current_y - cond_mean
                    quad_form = alpha**2 * (1 / cov_ygivenx)
                    neg_log_lik += 0.5 * (log_det + quad_form)

                    continue

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                L = torch.linalg.cholesky(cov_matrix)
                L11 = L[:1, :1]
                L12 = torch.zeros(L[:1, 1:].shape)
                L21 = L[1:, :1]
                L22 = L[1:, 1:]
                L11_inv = torch.linalg.inv(L11)
                L22_inv = torch.linalg.inv(L22)

                L_inv = torch.block_diag(L11_inv, L22_inv)
                L_inv[1:, :1] = -torch.matmul(L22_inv, L21) @ L11_inv

                cov_yx = cov_matrix[0, 1:]

                tmp1 = torch.matmul(L_inv, locs)
                tmp2 = torch.matmul(torch.matmul(L_inv, locs).T, torch.matmul(L_inv, y_and_neighbors))
                tmp_for_beta = torch.matmul(tmp1.T, tmp1)
                beta = torch.linalg.solve(tmp_for_beta, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)

             
                if time_idx == 1:
                    self.cov_map[index] = {
                        'tmp_for_beta': tmp_for_beta,
                        'cov_xx_inv': cov_xx_inv,
                        'cov_matrix': cov_matrix,
                        'L_inv': L_inv,
                        'cov_ygivenx': cov_ygivenx,
                        'cond_mean_tmp': cond_mean_tmp,
                        'log_det': log_det,
                        'locs': locs
                    }

        return neg_log_lik    
    



In [ ]:

# Choose your optimizer
params = [24.56, 2.8,  -0.0138, -0.0138, 0.301, 2.12]
params = [20, 8.25, 5.25, 0.2, 0.2, 0.2]

params = [20, 2, 2, 0.01, -0.1, 0.14, 3]

params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)

instance = matern_advec_beta_torch_vecchia(analysis_data_map, params, nns_map, mm_cond_number)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer = optim.Adam([params],lr=0.01, betas=(0.9, 0.8), eps=1e-8)
scheduler = StepLR(optimizer, step_size=20, gamma=0.9)

# Example function to compute out1
def compute_out1(params):
    # Compute the output using your function
    # nll = instance.full_likelihood(params, instance.matern_advec_beta_cov )
    nll = instance.vecchia_like_local_computer(params, instance.matern_cov_ani )

    return nll

# Training loop
prev_loss = float('inf')
tol = 1e-4  # Convergence tolerance
for epoch in range(3000):  # Number of epochs
    optimizer.zero_grad()  # Zero the gradients 
    
    loss = compute_out1(params)
    loss.backward()  # Backpropagate the loss
    
    # Print gradients and parameters every 10th epoch
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    # print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    optimizer.step()  # Update the parameters
    scheduler.step()
    # Check for convergence
    if abs(prev_loss - loss.item()) < tol:
        print(f"Converged at epoch {epoch}")
        break
    
    prev_loss = loss.item()

print('Training complete.') 

NameError: name 'optim' is not defined

Try my package oop using full likelihood 

In [24]:

params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)

instance = kernels.model_fitting(
    smooth=0.5,
    input_map=analysis_data_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number
)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer, scheduler = instance.optimizer_fun( params, lr=0.01, betas=(0.9, 0.8), eps=1e-8, step_size=20, gamma=0.9)    
instance.run_full(params, optimizer,scheduler, epochs=3000)

Epoch 1, Gradients: [  1.9402208  -8.884079   -8.339249   -2.2376022 -10.631287  126.89653
   8.968163 ]
 Loss: 628.3262939453125, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]
Epoch 101, Gradients: [ 0.40051484 -4.172155   -3.8580322  -0.147995   -0.50027466 -2.6572876
  3.629163  ]
 Loss: 598.65771484375, Parameters: [23.563217    2.761465    2.7625687   0.10603883  0.04937236  0.10878895
  2.5003712 ]
Epoch 201, Gradients: [-0.08677709 -2.0934935  -1.3359222  -0.050354   -0.02059937 -0.6693115
  0.93575895]
 Loss: 594.40234375, Parameters: [23.07261     3.2624993   3.2710943   0.04107396  0.075522    0.12459052
  1.9889146 ]
Epoch 301, Gradients: [-0.2727673  -1.3088493  -0.4664917  -0.02841187 -0.04309082 -0.4633789
 -0.03971767]
 Loss: 593.5638427734375, Parameters: [23.329903    3.556881    3.5775445   0.03232866  0.07745279  0.12197667
  1.9365772 ]
Epoch 401, Gradients: [-0.3030101  -1.0736284  -0.2778473   0.00257111  0.00158691 -0

[array([23.700626  ,  3.931397  ,  3.9574375 ,  0.03695429,  0.0769494 ,
         0.11098192,  2.1335363 ], dtype=float32),
 592.9525146484375]